In [ ]:
# MODELS MODULE

import operator


class RefPath:
    INITIAL = "#/info/x-clusters/"
    WORKER_NODES = "worker-nodes"
    PODS = "pods"
    CONTAINERS = "containers"
    INFO = "info"
    X_CLUSTERS = "x-clusters"
    X_LOCATION = "x-location"
    REF = "$ref"
    PATHS = "paths"

    def __init__(self, *arg):
        if len(arg) == 1:
            paths = arg[0].split("#/info/x-clusters/")
            parts = paths[1].split("/")
            self.cluster = parts[0]
            self.worker_node = parts[2]
            self.pod_name = parts[4]
            self.container_name = parts[6]
        else:
            self.cluster = arg[0]
            self.worker_node = arg[1]
            self.pod_name = arg[2]
            self.container_name = arg[3]

    @property
    def full_path(self):
        return f"{RefPath.INITIAL}{self.cluster}/{RefPath.WORKER_NODES}/{self.worker_node}/{RefPath.PODS}/" \
               f"{self.pod_name}/{RefPath.CONTAINERS}/{self.container_name}"


class Method:
    def __init__(self, path_name, method_name, ref_path, load, schema_name, full_method):
        self.path_name = path_name
        self.method_name = method_name
        self.ref_path = ref_path
        self.load = load
        self.schema_name = schema_name
        self.full_method = full_method
        self.contribution = 0

    def __str__(self):
        return self.path_name + ": " + self.method_name + " (" + self.ref_path.full_path + ") " + str(
            self.load) + " <-> " + self.schema_name


class Component:
    def __init__(self, name, load, full_component, ref_path=None, is_new=False):
        self.name = name
        self.load = load
        self.full_component = full_component
        self.ref_path = ref_path
        self.is_new = is_new

    def __str__(self):
        return self.name + " ---- " + str(self.load) + " (" + self.ref_path + ") " + "New ? " + str(self.is_new)


class Group:
    def __init__(self, parent_load, sum_of_loads, components):
        self.parent_load = parent_load
        self.sum_of_loads = sum_of_loads
        self.components = components

    def get_contributed_components(self):
        components = []
        for component in self.components:
            component.contribution = (component.load / self.sum_of_loads) * self.parent_load
            components.append(component)
        return sorted(components, key=operator.attrgetter("contribution"))


class Cluster(Component):
    def __init__(self, name, load, full_component, ref_path, is_new=False):
        super().__init__(name, load, full_component, ref_path, is_new)
        self.is_cluster = True

    @property
    def worker_nodes(self):
        worker_nodes = []
        for wn in self.full_component["worker-nodes"].values():
            ref_path = self.ref_path + "/worker-nodes/" + wn["name"]
            worker_nodes.append(WorkerNode(wn["name"], wn["metrics"]["load"], wn, ref_path))
        return worker_nodes


class WorkerNode(Component):
    def __init__(self, name, load, full_component, ref_path, is_new=False):
        super().__init__(name, load, full_component, ref_path, is_new)
        self.is_worker_node = True

    @property
    def pods(self):
        pods = []
        for pod in self.full_component["pods"].values():
            ref_path = self.ref_path + "/pods/" + pod["name"]
            pods.append(Pod(pod["name"], pod["metrics"]["load"], pod, ref_path))
        return pods


class Pod(Component):
    def __init__(self, name, load, full_component, ref_path, is_new=False):
        super().__init__(name, load, full_component, ref_path, is_new)
        self.is_pod = True
        self.contribution = 0

    @property
    def containers(self):
        containers = []
        for container_name, container in self.full_component["containers"].items():
            ref_path = self.ref_path + "/containers/" + container_name
            containers.append(Container(container_name, container["metrics"]["load"], container, ref_path))
        return containers


class Container(Component):
    def __init__(self, name, load, full_component, ref_path, is_new=False):
        super().__init__(name, load, full_component, ref_path, is_new)
        self.is_container = True


class PodGroup(Group):
    def __init__(self, parent_load, sum_of_loads, components):
        super().__init__(parent_load, sum_of_loads, components)
        self.is_pod_group = True

    def __str__(self):
        return "PodGroup sum_of_loads: " + str(self.sum_of_loads)


class ContainerGroup(Group):
    def __init__(self, parent_load, sum_of_loads, components):
        super().__init__(parent_load, sum_of_loads, components)
        self.is_container_group = True

    def __str__(self):
        return "ContainerGroup sum_of_loads: " + str(self.sum_of_loads)


class MethodGroup(Group):
    def __init__(self, parent_load, sum_of_loads, components):
        super().__init__(parent_load, sum_of_loads, components)
        self.is_method_group = True

    def __str__(self):
        return "ContainerGroup sum_of_loads: " + str(self.sum_of_loads)


In [ ]:
import copy
import os


def _gen_dict_extract(key, var):
    if hasattr(var, 'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):
                for result in _gen_dict_extract(key, v):
                    yield result
            elif isinstance(v, (list)):
                for d in v:
                    for result in _gen_dict_extract(key, d):
                        yield result


def _get_schema_only(references):
    saved_schema = 'default'
    for reference in references:
        schema = reference.split("#/components/schemas/")
        try:
            saved_schema = schema[1]
        except IndexError:
            pass

    return saved_schema


def _get_schemas_only(references):
    saved_schemas = []
    for reference in references:
        schema = reference.split("#/components/schemas/")
        try:
            saved_schemas.append(schema[1])
        except IndexError:
            pass

    return saved_schemas


def _join_components(threshold, components):
    joined_components = []
    remaining_components = []
    collective_contribution = 0
    for i in range(len(components)):
        component = components[i]
        collective_contribution += component.contribution

        if collective_contribution < threshold:
            joined_components.append(component)
        else:
            remaining_components.append(component)

    return joined_components, remaining_components


def _derive_template_components(template):
    clusters = []
    cls = template[RefPath.INFO][RefPath.X_CLUSTERS]

    for cl in cls.values():
        ref_path = RefPath.INITIAL + cl["name"]
        clusters.append(Cluster(cl["name"], cl["metrics"]["load"], cl, ref_path))

    methods = []
    data_paths = template[RefPath.PATHS]
    for path_name, path in data_paths.items():
        for method_name, method in path.items():
            ref_path = RefPath(method[RefPath.X_LOCATION][RefPath.REF])

            all_references = list(set(_gen_dict_extract('$ref', method)))
            schema_name = _get_schema_only(all_references)

            methods.append(
                Method(path_name, method_name, ref_path, method["x-metrics"]["load"], schema_name, method))
    return clusters, methods


def clean_template(template):
    t_clusters, t_methods = _derive_template_components(template)
    clusters_template = template[RefPath.INFO][RefPath.X_CLUSTERS]

    for cluster in t_clusters:
        for worker_node in cluster.worker_nodes:
            filtered_wn_methods = list(
                filter(lambda method: method.ref_path.worker_node == worker_node.name, t_methods))
            if not filtered_wn_methods:
                print("Deleting worker node:", worker_node.ref_path)
                del clusters_template[cluster.name][RefPath.WORKER_NODES][worker_node.name]
                # continue
            for pod in worker_node.pods:
                filtered_pod_methods = list(filter(
                    lambda method: method.ref_path.worker_node == worker_node.name and
                                   method.ref_path.pod_name == pod.name, t_methods))
                if not filtered_pod_methods:
                    print("Deleting pod:", pod.ref_path)
                    del clusters_template[cluster.name][RefPath.WORKER_NODES][worker_node.name][RefPath.PODS][pod.name]
                    # continue
                for container in pod.containers:
                    filtered_container_methods = list(filter(
                        lambda method: method.ref_path.worker_node == worker_node.name and method.ref_path.pod_name == pod.name and method.ref_path.container_name == container.name,
                                                             t_methods))
                    if not filtered_container_methods:
                        print("Deleting container:", container.ref_path)
                        del clusters_template[cluster.name][RefPath.WORKER_NODES][worker_node.name][RefPath.PODS][
                            pod.name][RefPath.CONTAINERS][container.name]


def reorder_template(template):
    c_template = copy.deepcopy(template)
    # cc_template = copy.deepcopy(template)

    c_clusters_template = c_template[RefPath.INFO][RefPath.X_CLUSTERS]
    cc_clusters_template = template[RefPath.INFO][RefPath.X_CLUSTERS]

    c_clusters, c_methods = _derive_template_components(c_template)

    for r_cl in range(len(c_clusters)):
        cl = c_clusters[r_cl]
        cl_name = "cl" + str(r_cl + 1)
        if cl.name == cl_name:
            pass
        else:
            cl_value = c_clusters_template[cl.name]
            cl_value['name'] = cl_name
            cc_clusters_template[cl_name] = cl_value

            cl_methods = filter(lambda m: m.ref_path.cluster == cl.name, c_methods)
            for method in cl_methods:
                print("Changed from:", method.ref_path.full_path)
                method.ref_path.cluster = cl_name
                method.full_method['x-location'][RefPath.REF] = method.ref_path.full_path
                print("To:", method.ref_path.full_path)

                template[RefPath.PATHS][method.path_name][method.method_name] = method.full_method

            del c_clusters_template[cl.name]
            cl.name = cl_name

        for r_wn in range(len(cl.worker_nodes)):
            wn = cl.worker_nodes[r_wn]
            wn_name = "wn" + str(r_wn + 1)
            if wn.name == wn_name:
                pass
            else:
                wn_value = c_clusters_template[cl.name][RefPath.WORKER_NODES][wn.name]
                wn_value['name'] = wn_name

                cc_clusters_template[cl.name][RefPath.WORKER_NODES][wn.name] = wn.full_component

                wn_methods = filter(
                    lambda m: m.ref_path.cluster == cl.name and m.ref_path.worker_node == wn.name,
                    c_methods)
                for method in wn_methods:
                    print("Changed from:", method.ref_path.full_path)
                    method.ref_path.worker_node = wn_name
                    method.full_method['x-location'][RefPath.REF] = method.ref_path.full_path
                    print("To:", method.ref_path.full_path)

                    template[RefPath.PATHS][method.path_name][method.method_name] = method.full_method

                del c_clusters_template[cl.name][RefPath.WORKER_NODES][wn.name]
                wn.name = wn_name

            for r_pod in range(len(wn.pods)):
                pod = wn.pods[r_pod]
                pod_name = "pod" + str(r_pod + 1)

                if pod.name == pod_name:
                    pass
                else:
                    pod_value = c_clusters_template[cl.name][RefPath.WORKER_NODES][wn.name][RefPath.PODS][pod.name]
                    pod_value['name'] = pod_name

                    cc_clusters_template[cl.name][RefPath.WORKER_NODES][wn.name][RefPath.PODS][pod_name] = pod_value

                    pod_methods = filter(lambda
                                             m: m.ref_path.cluster == cl.name and m.ref_path.worker_node == wn.name and m.ref_path.pod_name == pod.name,
                                         c_methods)
                    for method in pod_methods:
                        print("Changed from:", method.ref_path.full_path)
                        method.ref_path.pod_name = pod_name
                        method.full_method['x-location'][RefPath.REF] = method.ref_path.full_path
                        print("To:", method.ref_path.full_path)

                        template[RefPath.PATHS][method.path_name][method.method_name] = method.full_method

                    del cc_clusters_template[cl.name][RefPath.WORKER_NODES][wn.name][RefPath.PODS][pod.name]
                    pod.name = pod_name
                for r_container in range(len(pod.containers)):
                    container = pod.containers[r_container]
                    container_name = "c" + str(r_container + 1)
                    if container.name == container_name:
                        pass
                    else:
                        container_value = c_clusters_template[cl.name][RefPath.WORKER_NODES][wn.name][
                            RefPath.PODS][pod.name][RefPath.CONTAINERS][container.name]
                        container_value['id'] = container_name

                        cc_clusters_template[cl.name][RefPath.WORKER_NODES][wn.name][RefPath.PODS][pod.name][
                            RefPath.CONTAINERS][container_name] = container_value

                        containers_methods = filter(lambda m: m.ref_path.cluster == cl.name
                                                              and m.ref_path.worker_node == wn.name
                                                              and m.ref_path.pod_name == pod.name
                                                              and m.ref_path.container_name == container.name,
                                                    c_methods)
                        for method in containers_methods:
                            print("Changed from:", method.ref_path.full_path)
                            method.ref_path.container_name = container_name
                            method.full_method['x-location'][RefPath.REF] = method.ref_path.full_path
                            print("To:", method.ref_path.full_path)

                            template[RefPath.PATHS][method.path_name][method.method_name] = method.full_method

                        del cc_clusters_template[cl.name][RefPath.WORKER_NODES][wn.name][RefPath.PODS][pod.name][
                            RefPath.CONTAINERS][container.name]
                        container.name = container_name


def get_latest_filetag(dir_path):
    """get the number of latest configuration file"""

    dir_size = (len(os.listdir(dir_path)))
    file_tag = int(dir_size / 2)
    if file_tag < 10:
        file_tag = str("0" + str(file_tag))
    return file_tag


In [ ]:
# CONSTANTS MODULES

MAX_WN_LOAD = 50
MIN_WN_LOAD = 20
MAX_POD_LOAD = 50
MIN_POD_LOAD = 30
DEFAULT_SCHEMA_NAME = 'default'
POD_LEVEL = "pod"
CL_LEVEL = "cluster"
WN_LEVEL = "worker-node"
SCHEMA_LEVEL = "x-storage-level"

In [ ]:
import json

with open("01config.json", "r") as file:
    config = json.load(file)

In [ ]:
import copy

def get_methods_by_ref_path(methods, ref_path):
    ref_path_methods = {}
    for method in methods:
        if method.ref_path.full_path == ref_path:
            method_path = ref_path_methods.setdefault(method.path_name, {})
            method_path[method.method_name] = method.full_method
    return ref_path_methods

def get_methods_by_schema(methods, schema):
    schema_methods = {}
    for method in methods:
        if method.schema_name == schema:
            method_path = schema_methods.setdefault(method.path_name, {})
            method_path[method.method_name] = method.full_method
    return schema_methods

def to_template_dto(config):
    copied_template = copy.deepcopy(config)
    clusters, methods = _derive_template_components(copied_template)

    for cluster in clusters:    
        for wn in cluster.worker_nodes:        
            for pod in wn.pods:            
                for container in pod.containers:
                    container.full_component['methods'] = get_methods_by_ref_path(methods, container.ref_path)


    for method in methods:
        schemas = copied_template.setdefault('schemas', {})
        schemas[method.schema_name] = get_methods_by_schema(methods, method.schema_name)
    return copied_template

In [ ]:
to_template_dto(config)

In [ ]:
copied_template